In [1]:
import sqlite3
import sqlite_vec
import os

# 确保data目录存在
os.makedirs('./data', exist_ok=True)

# 创建数据库文件
db_path = './data/vector_database.sqlite'
db = sqlite3.connect(db_path)

# 启用扩展加载
db.enable_load_extension(True)

# 加载sqlite_vec扩展
try:
    sqlite_vec.load(db)
    print("SQLite vector extension loaded successfully")
except Exception as e:
    print(f"Failed to load vector extension: {e}")

# 读取并处理SQL文件
sql_path = './data/test_vec.sql'

if os.path.exists(sql_path):
    with open(sql_path, 'r', encoding='utf-8') as f:
        sql_content = f.read()
    
    # 移除SQLite命令行指令（以点开头的行）
    sql_lines = sql_content.split('\n')
    filtered_lines = [line for line in sql_lines if not line.strip().startswith('.')]
    clean_sql = '\n'.join(filtered_lines)
    
    # 按分号分割SQL语句
    sql_statements = [stmt.strip() for stmt in clean_sql.split(';') if stmt.strip()]
    
    # 逐个执行SQL语句
    for i, sql_stmt in enumerate(sql_statements):
        try:
            if sql_stmt.upper().startswith('SELECT'):
                # 对于SELECT语句，执行并显示结果
                cursor = db.execute(sql_stmt)
                results = cursor.fetchall()
                print(f"\n--- Query {i+1} Results ---")
                print(f"SQL: {sql_stmt[:50]}...")
                for row in results[:5]:  # 只显示前5行
                    print(row)
                if len(results) > 5:
                    print(f"... and {len(results) - 5} more rows")
            else:
                # 对于其他语句，直接执行
                db.execute(sql_stmt)
                print(f"Executed: {sql_stmt[:50]}...")
        except sqlite3.Error as e:
            print(f"Error executing SQL: {e}")
            print(f"SQL statement: {sql_stmt[:100]}...")
    
    # 提交所有更改
    db.commit()
    print("\nAll SQL statements executed and committed successfully")
    
else:
    print(f"SQL file not found: {sql_path}")

# 验证数据库创建成功
print("\n--- Database Verification ---")
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f"Tables created: {[table[0] for table in tables]}")

# 检查每个表的记录数
for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    count = cursor.fetchone()[0]
    print(f"Table '{table_name}': {count} records")

# 关闭数据库连接
db.close()
print("\nDatabase connection closed")

SQLite vector extension loaded successfully
Executed: -- 加载vec0扩展

-- 1. 创建产品表（8维向量）
CREATE VIRTUAL TABL...
Executed: -- 插入30条产品数据（含电子产品、服装、食品等）
INSERT INTO products(ro...
Executed: -- 2. 创建用户表（16维向量）
CREATE VIRTUAL TABLE users USIN...
Executed: -- 插入30条用户数据（含随机生成的16维向量）
INSERT INTO users(rowid,...
Executed: -- 3. 创建地理位置表（4维向量）
CREATE VIRTUAL TABLE locations...
Executed: -- 插入30条地理位置数据（全球主要城市坐标）
INSERT INTO locations(row...

--- Query 7 Results ---
SQL: SELECT rowid, username, age, gender FROM users...
(1, 'AliceSmith', 28, 'female')
(2, 'BobJohnson', 35, 'male')
(3, 'CharlieBrown', 22, 'male')
(4, 'DianaLee', 31, 'female')
(5, 'EvaChen', 27, 'female')
... and 25 more rows

--- Query 8 Results ---
SQL: SELECT rowid, coordinates, place_name, city, count...
(1, b'\xd9\xce\xe8B\xf6\xa8\x1fB\x00\x00HB\n\xd7#<', '天安门广场', '北京', '中国')
(2, b'-\xf2\xf2B\n\xd7\xf9A\x00\x00 A\xcd\xccL=', '外滩', '上海', '中国')
(3, b'+\x87\xe2B1\x08\xb9A\x00\x00\xa0B\n\xd7\xa3<', '广州塔', '广州', '中国')
(4, b'\x87V\xe4BP\x8